<h3>Language modelling</h3>
<p>Konrad Przewłoka</p>

<h4>Imports</h4>

In [1]:
#!pip install torch
#!pip install transformers
#!pip install sacremoses
from pprint import pprint
import os
import random
import math
import collections
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import torch
import transformers

C:\Users\KPR\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h4>Loading Twitter TwHIN-BERT model<h4>

In [27]:
tokenizer_tw = transformers.AutoTokenizer.from_pretrained("Twitter/twhin-bert-base")
model_tw = transformers.AutoModelForMaskedLM.from_pretrained("Twitter/twhin-bert-base")

<h4>Loading XLM-RoBERTa large model<h4>

In [28]:
tokenizer_xlm = transformers.AutoTokenizer.from_pretrained("xlm-roberta-large")

model_xlm = transformers.AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

<h4>Loading polish BERT cased model</h4>

In [29]:
tokenizer_bert_cased = transformers.AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")

model_bert_cased = transformers.AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<h4>Utility array of models</h4>

In [30]:
models = [
    {'name':"TwHIN-BERT", 'model':model_tw,'tokenizer':tokenizer_tw},
    {'name':"xlm roberta large", 'model':model_xlm,'tokenizer':tokenizer_xlm},
    {'name':"Polish BERT cased", 'model':model_bert_cased,'tokenizer':tokenizer_bert_cased}
]

<h4>Polish cases checking</h4>

In [31]:
def unmask_sentece(model,sentence,num_tokens):
    sentence = sentence.replace('[MASK]',model['tokenizer'].mask_token)
    generator = transformers.pipeline(task="fill-mask", model=model['model'], tokenizer=model['tokenizer'],top_k=num_tokens)
    top_tokens = generator(sentence)
    #print(top_tokens)
    return top_tokens

example_sentences=[
    "Warszawa to największe [MASK].",
    "Marek bardzo długo szukał [MASK].",
    "Prezenty podobały się [MASK].",
    "Po wejściu na dach można obejrzeć [MASK].",
    "Tomasz postanowił pojechać do domu z [MASK].",
    "Anna od dawna marzyła o [MASK].",
    "Witaj [MASK]!"
]

for model in models:
    print(model['name'])
    for sentence in example_sentences:
        print(sentence)
        for token in unmask_sentece(model, sentence,3):
            print(sentence.replace("[MASK]",token['token_str'])+" score:"+str(token['score']))

TwHIN-BERT
Warszawa to największe [MASK].
Warszawa to największe miasto. score:0.736761212348938
Warszawa to największe miasta. score:0.05466261878609657
Warszawa to największe miejsce. score:0.03227910026907921
Marek bardzo długo szukał [MASK].
Marek bardzo długo szukał pomocy. score:0.10109560191631317
Marek bardzo długo szukał syna. score:0.03973833844065666
Marek bardzo długo szukał klubu. score:0.024326728656888008
Prezenty podobały się [MASK].
Prezenty podobały się mi. score:0.3716319799423218
Prezenty podobały się najbardziej. score:0.0731700211763382
Prezenty podobały się bardzo. score:0.0708843320608139
Po wejściu na dach można obejrzeć [MASK].
Po wejściu na dach można obejrzeć mecz. score:0.13820107281208038
Po wejściu na dach można obejrzeć telewizor. score:0.04552201181650162
Po wejściu na dach można obejrzeć film. score:0.04437772184610367
Tomasz postanowił pojechać do domu z [MASK].
Tomasz postanowił pojechać do domu z pracy. score:0.07852048426866531
Tomasz postanowił po

<h4>Testing long-range relationships</h4>

In [37]:
example_sentences_relationships=[
    "Anna wybrała się do sklepu, gdzie [MASK] mleko.",
    "Marek wrócił do domu i natychmiast [MASK] na kanapie.",
    "To drzewo [MASK] 50 lat zanim spłonęło.",
    "Tomek postanowił odwiedzić Marka, ponieważ bardzo go [MASK].",
    "Tomek postanowił odwiedzić Marka, ponieważ bardzo [MASK] lubił."    
]
for model in models:
    print(model['name'])
    for sentence in example_sentences_relationships:
        print(sentence)
        for token in unmask_sentece(model, sentence,2):
            print(sentence.replace("[MASK]",token['token_str'])+" score:"+str(token['score']))

TwHIN-BERT
Anna wybrała się do sklepu, gdzie [MASK] mleko.
Anna wybrała się do sklepu, gdzie jest mleko. score:0.5127535462379456
Anna wybrała się do sklepu, gdzie było mleko. score:0.1643742471933365
Marek wrócił do domu i natychmiast [MASK] na kanapie.
Marek wrócił do domu i natychmiast jest na kanapie. score:0.19211305677890778
Marek wrócił do domu i natychmiast wraca na kanapie. score:0.09422203153371811
To drzewo [MASK] 50 lat zanim spłonęło.
To drzewo . 50 lat zanim spłonęło. score:0.12717671692371368
To drzewo było 50 lat zanim spłonęło. score:0.07814445346593857
Tomek postanowił odwiedzić Marka, ponieważ bardzo go [MASK].
Tomek postanowił odwiedzić Marka, ponieważ bardzo go kocha. score:0.28028708696365356
Tomek postanowił odwiedzić Marka, ponieważ bardzo go lubi. score:0.16457785665988922
Tomek postanowił odwiedzić Marka, ponieważ bardzo [MASK] lubił.
Tomek postanowił odwiedzić Marka, ponieważ bardzo go lubił. score:0.8882428407669067
Tomek postanowił odwiedzić Marka, ponieważ

<h4>Checking real-world knowledge</h4>

In [35]:
example_sentences_relationships=[
    "Tomek studiuje polonistykę na [MASK].",
    "[MASK] wschodzi o ósmej i zachodzi około szesnastej.",
    "Nocami wpatrywał się w [MASK] Saturna.",
    "[MASK] leży pomiędzy Chinami a Rosją."
]
for model in models:
    print(model['name'])
    for sentence in example_sentences_relationships:
        print(sentence)
        for token in unmask_sentece(model, sentence,2):
            print(sentence.replace("[MASK]",token['token_str'])+" score:"+str(token['score']))

TwHIN-BERT
Tomek studiuje polonistykę na [MASK].
Tomek studiuje polonistykę na uczelni. score:0.10560335218906403
Tomek studiuje polonistykę na żywo. score:0.08208702504634857
[MASK] wschodzi o ósmej i zachodzi około szesnastej.
Czas wschodzi o ósmej i zachodzi około szesnastej. score:0.0806540995836258
Dzień wschodzi o ósmej i zachodzi około szesnastej. score:0.055803991854190826
Nocami wpatrywał się w [MASK] Saturna.
Nocami wpatrywał się w twarz Saturna. score:0.2190801501274109
Nocami wpatrywał się w oczy Saturna. score:0.05704287439584732
[MASK] leży pomiędzy Chinami a Rosją.
Polska leży pomiędzy Chinami a Rosją. score:0.2131042778491974
Putin leży pomiędzy Chinami a Rosją. score:0.04738715663552284
xlm roberta large
Tomek studiuje polonistykę na [MASK].
Tomek studiuje polonistykę na uczelni. score:0.11222496628761292
Tomek studiuje polonistykę na UM. score:0.068116195499897
[MASK] wschodzi o ósmej i zachodzi około szesnastej.
słońce wschodzi o ósmej i zachodzi około szesnastej. sc

<h4>Answers</h4>
<h5>I</h5>
<p>The model to produce best results was Polish BERT cased, available at: https://huggingface.co/dkleczek/bert-base-polish-cased-v1</p>
<h5>II</h5>
<p>All of tested models seem to capture at least a rudimentary level of polish grammar</p>
<h5>III</h5>
<p>All models display capability in terms of capturing long-distance relationships. That being said, the TwHIN-BERT model results are of lower quality compared to others. The polish BERT cased model seems to offer the best results in terms of capturing long-distance relationships.</p>
<h5>IV</h5>
<p>All tested models seem to capture some real-world knowledge although they make some significant mistakes in this category. Those mistakest showcase greatly that tested models don't have a real understanding of the language as most mistakes made in the real-world knowledge were correct in terms of grammar but completely illogical ex. "Putin leży pomiędzy Chinami a Rosją.". In some test cases however I am willing to be lenient since knowledge of certain things isn't an universal property of every human. For example we may ask ourselves if we questioned 100 randomly chosen correspondents, how many of them would point to Mongolia as the country landlocked between China and Russia.</p>
<h5>V</h5>
<p>The most striking mistakes by test:
    Polish cases:
    <ul>
    <li> Witaj wszystkim! - TwHIN-BERT
    <li> Anna od dawna marzyła o ciebie. - TwHIN-BERT
    <li> Witaj cie! - xlm roberta
    </ul>
    Long-range relationships:
    <ul>
    <li> Marek wrócił do domu i natychmiast wraca na kanapie. - TwHIN-BERT
    <li> To drzewo . 50 lat zanim spłonęło. - TwHIN-BERT
    <li> Tomek postanowił odwiedzić Marka, ponieważ bardzo ich lubił. - TwHIN-BERT
    <li> Marek wrócił do domu i natychmiast siedzi na kanapie. - xlm roberta
    <li> Anna wybrała się do sklepu, gdzie kupić mleko. - polish BERT
    <li> Tomek postanowił odwiedzić Marka, ponieważ bardzo ją lubił. - polish BERT
    </ul>
    Real-world knowledge:
    <ul>
    <li> Tomek studiuje polonistykę na żywo. - TwHIN-BERT
    <li> Czas wschodzi o ósmej i zachodzi około szesnastej. - TwHIN-BERT
    <li> Nocami wpatrywał się w twarz Saturna. - TwHIN-BERT
    <li> Nocami wpatrywał się w oczy Saturna. - TwHIN-BERT
    <li> Polska leży pomiędzy Chinami a Rosją. - TwHIN-BERT
    <li> Putin leży pomiędzy Chinami a Rosją. - TwHIN-BERT
    <li> Polska leży pomiędzy Chinami a Rosją. - xlm roberta
    <li> Ukraina leży pomiędzy Chinami a Rosją. - xlm roberta
    <li> Nocami wpatrywał się w gwiazdy Saturna. - polish BERT
    <li> Polska leży pomiędzy Chinami a Rosją. - polish BERT
    </ul>
   Worst mistakes were made by the TwHIN-BERT model. At the same time that most striking and numerous mistakes were made in the real-world knowledge category, this best showcases that tested models present little "understanding" of the concepts that language communicates and just "pretend" to understand the information given in sentences.
</p>